### SWC/Gatsby Boot Camp Python tutorial 
#### unittest modul

Why should we do testing?

*  Make sure our code behave properly under different conditions 
    (`5/2` vs `5.0/2`, `log(0)`, `1/0`)

*  Regression testing: use bugs found in the past to make sure the
    current version do no reproduce the same bug

*  Before shipping/publishing the code, a prepared test suite will 
    let the user know if the package they downloaded behave the same 
    way you expected it to. 

Basic usage:

1.  Define a class derived from unittest
2.  Fill the class with functions that start with test_
3.  You run the tests by placing unittest.main() in your file

See the unittest_tutorial.py file for basic usage. 

Let's first run the .py file 

In [4]:
! type unittest_tutorial.py

'''
######################################

SWC/Gatsby Boot Camp Python tutorial
uniitest module

######################################

    Why should we do testing?

1.  Make sure our code behave properly under different conditions 
    (5/2 vs 5.0/2, log(0), 1/0)

2.  Regression testing: use bugs found in the past to make sure the
    current version do no reproduce the same bug

3.  Before shipping/publishing the code, a prepared test suite will 
    let the user know if the package they downloaded behave the same 
    way you expected it to. 



    Basic usage:

1.  Define a class derived from unittest
2.  Fill the class with functions that start with test_
3.  You run the tests by placing unittest.main() in your file
    
'''



import unittest

# 1.  Define a class derived from unittest
class TestStringMethods(unittest.TestCase):

# 2.  Fill the class with functions that start with test_
    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isu

In [5]:
! python -m unittest unittest_tutorial

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


### Let's try a more realistic test scenario
Let's define a class called `supstr` derived from `str` with some fancy methods, and then test it using `unittest`

First, copy the original tests for methods of `str` as we don't want our `supstr` class to harm `str` (which won't happen in this case, but let's presume that you wrote `str` class yourself and also wrote the test cases below already)

In [12]:
import unittest

# 1.  Define a class derived from unittest
class TestStringMethods(unittest.TestCase):

# 2.  Fill the class with functions that start with test_
    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)
# 3.  You run the tests by placing unittest.main() in your file

Second, just define `supstr` that just inherits from `str`

In [13]:
class supstr(str):
    pass

Define test classes on `supstr`. Note that since we would like to test `supstr`, we have to create it in our test cases. This can be done using `setUp` which is a method that is run before any tests were taken. This is used to set up some variable or open files that are used for the test cases. 

At the end of the test, we can use `tearDown` to undo any changes to the system which can include closing files or connections. 

In [14]:
class TestSupStringMethods(unittest.TestCase):

    def setUp(self):
        self.ss = supstr('superstring')

    def test_upper(self):
        self.assertEqual(self.ss.upper(), 'SUPERSTRING') 

    def tearDown(self):
        del self.ss


The code below shows a more elaborate way of running a test with finer control using `TestLoader` and `TestSuite`

In [15]:
# define a loader object
loader = unittest.TestLoader()
# define a suite object that is going to hold all tests
suite = unittest.TestSuite()

The following line takes a test class and returns a test suite object

In [16]:
suite = loader.loadTestsFromTestCase(TestStringMethods)

We can add tests by addTests

In [17]:
suite.addTests(loader.loadTestsFromTestCase(TestSupStringMethods))
unittest.TextTestRunner(verbosity=2).run(suite)

test_isupper (__main__.TestStringMethods) ... ok
test_split (__main__.TestStringMethods) ... ok
test_upper (__main__.TestStringMethods) ... ok
test_upper (__main__.TestSupStringMethods) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.006s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [18]:
class supstr(str):
    def __div__(self, s):
        return self.count(s)

Now we could have written a test suite (class) that contains multiple data, but we can also add a function as a test case into the suite

In [8]:
def testSupStrDivide():
    ss1 = supstr('baaaa')
    ss2 = supstr('aa')
    assert ss1/ss2 == 3
    
testcase = unittest.FunctionTestCase(testSupStrDivide)

suite.addTest(testcase)

In [9]:
unittest.TextTestRunner(verbosity=2).run(suite)

test_isupper (__main__.TestStringMethods) ... ok
test_split (__main__.TestStringMethods) ... ok
test_upper (__main__.TestStringMethods) ... ok
test_upper (__main__.TestSupStringMethods) ... ok
unittest.case.FunctionTestCase (testSupStrDivide) ... FAIL

FAIL: unittest.case.FunctionTestCase (testSupStrDivide)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-bbe7918a8dd1>", line 4, in testSupStrDivide
    assert ss1/ss2 == 3
AssertionError

----------------------------------------------------------------------
Ran 5 tests in 0.008s

FAILED (failures=1)


<unittest.runner.TextTestResult run=5 errors=0 failures=1>

As expected, there is an error... turns out that `str.count()` does not deal with overlapping sequences. Let's not be so lazy and do it the hard way

In [10]:
class supstr(str):
    def __div__(self, s):
        finished = False
        start = 0
        count = 0
        while not finished:
            idx = self.find(s, start)
            if idx == -1:
                finished = True
            else:
                count += 1
                start = idx + 1
        return count

Now run the same test again

In [11]:
unittest.TextTestRunner(verbosity=2).run(suite)

test_isupper (__main__.TestStringMethods) ... ok
test_split (__main__.TestStringMethods) ... ok
test_upper (__main__.TestStringMethods) ... ok
test_upper (__main__.TestSupStringMethods) ... ok
unittest.case.FunctionTestCase (testSupStrDivide) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.012s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

## Discover test cases

In real development, you are probably writing many test files (classes) all over the place in the directory. The `unittest.TestLoader.discovery()` function can be used to find all test files (test*.py) and return a test suite. This can also be achieved by the command line

    cd project_directory
    python -m unittest discover
    
for more details, see https://docs.python.org/2/library/unittest.html#test-discovery